In [1]:
import sys
sys.path.append('/pl/active/banich/studies/Relevantstudies/abcd/env/lib/python3.7/site-packages')
sys.path.append('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/analysis/')
from functions import *

In [2]:
include = pd.read_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/inclusion_ids.csv')[['ID']]

new_demos = readit("/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/abcd_demos.csv")

new_demos = (new_demos.rename(columns={'src_subject_id': 'ID', 'high.educ': 'high_educ', 'household.income': 'household_income'})
             .query('event_name == "baseline_year_1_arm_1"')
             #.dropna()
            )
new_demos['ID'] = new_demos['ID'].str.replace('_', '')

family = new_demos[['ID', 'rel_family_id']]

#family = pd.merge(include, family, on='ID')

dropped_siblings = family.groupby('rel_family_id').sample(n=1, random_state=10).reset_index(drop=True)[['ID']]

In [3]:
demos = new_demos[['ID', 'race_ethnicity', 'household_income', 'hisp', 'sex_at_birth', 'age', 'high_educ', 'married', 'abcd_site']]
demos = demos.fillna('NA')

demos[['household_income']] = demos[['household_income']].applymap(lambda x: x.replace('[', '').replace(']', ''))
print("Demographics: new_demos, demos")

full_demos = (readtxt('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/abcd_lpds01.txt')
              .rename(columns={'subjectkey': 'ID'}))
adversitycols = list(full_demos.filter(regex='_exp').columns)

def filt(df, cols, thresh):
    return df.loc[(df[cols] < thresh).all(axis=1)]

full_demos['ID'] = full_demos['ID'].str.replace('_', '')
full_demos_adv = full_demos[['ID'] + adversitycols].dropna()
full_demos_adv = full_demos_adv.drop_duplicates(subset ="ID", keep = "first")
full_demos_adv[adversitycols] = full_demos_adv[adversitycols].apply(pd.to_numeric, errors='coerce')
full_demos_adv = filt(full_demos_adv, adversitycols, 777)
full_demos_adv['adversity'] = full_demos_adv[adversitycols].sum(axis=1)
demos_adv = full_demos_adv[['ID', 'adversity']]
print("Adversity: demos_adv")

Demographics: new_demos, demos
Adversity: demos_adv


In [4]:
demos_baseline = (demos.merge(demos_adv , on='ID', how='outer')
 .drop_duplicates('ID', keep='first'))

In [5]:
demos_baseline.to_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/demos_baseline.csv', index=False)

In [6]:
dropped_siblings.to_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/dropped_siblings.csv', index=False)